## `establish_connection`

Establishes the connection to the database. Accepts a hash as input where the :adapter key must be specified with the name of a database adapter (in lower-case) example for regular databases (MySQL, PostgreSQL, etc

当调用一个不存在的方法时，Ruby会沿着祖先链向上查找这个方法。如果一直找到`BasicObject`还没有找到这个方法，则`Ruby`会回到祖先链底，调用`method_misssing`这个方法，这意味着幽灵方法的调用要慢于普通方法的调用，因为`Ruby`至少要完整的查找一遍祖先链

在大多数情况下，幽灵方法和普通方法的效率可以忽略不计。然而，在`Rails`中，属性方法的调用非常频繁。在`Rails 1`中，每一次对属性方法的调用都不得不遍历`ActiveRecord::Base`那么长的祖先链，因此效率非常的低。

In [2]:
class MyClass
  attr_accessor :my_attribute
  
  def set_attribute(n)
    my_attribute = n
  end
end

obj = MyClass.new
obj.set_attribute 10
p obj.my_attribute

nil


这个结果可能在你的意料之外。这是因为`self_attribute`的代码具有不确定性。`Ruby`不知道代码是想给局部变量赋值，还是调用名为`my_attribute=`的拟态方法。在没有确定答案的情况下，Ruby默认选择第一种方式，它定义了一个名为`my_attribute`的局部变量，该变量在复制完后就落在作用域之外了。

为了避免这个问题，给当前对象的属性赋值时，应该显性地使用`self`。

In [3]:
class MyClass
  def set_attribute(n)
    self.my_attribute = n
  end
end

obj.set_attribute 10
obj.my_attribute

10

自由方法（unbound method）更普通方法类似，不过它从最初定义它的类或者模块中脱离了。通过调用`Method#unbind`方法，可以把一个方法变成自由方法。你也可以直接调用`Module#instance_method`方法获得一个自由方法。下面是一个例子：

In [4]:
module MyModule
  def my_method
    42
  end
end

unbound = MyModule.instance_method(:my_method)
unbound.class

UnboundMethod

虽然不能调用`UnboundMethod`，但可以把它绑定到一个对象上，使之再次成为一个`Method`对象。具体的做法是使用`UnboundMehtod#bind`方法把`UnboundMethod`对象绑定到一个对象上。还可以把`UnboundMehtod`对象在传给`Module#define_method`方法，从而实现绑定。

In [5]:
String.send :define_method, :another_method, unbound
"abc".another_method

42

It's shorthand for tags.map(&:name.to_proc).join(' ')

If foo is an object with a to_proc method, then you can pass it to a method as &foo, which will call foo.to_proc and use that as the method's block.

The Symbol#to_proc method was originally added by ActiveSupport but has been integrated into Ruby 1.8.7. This is its implementation:

In [6]:
class Symbol
  def to_proc
    Proc.new do |obj, *args|
      obj.send self, *args
    end
  end
end

:to_proc

空指针和布尔值一起的时候会变得很奇怪

In [7]:
def calculate_initial_value
  puts "called: calculate_initial_value"
  false
end

b = nil
2.times do
  b ||= calculate_initial_value
end


called: calculate_initial_value
called: calculate_initial_value


2

空指针等价的`if`语句

In [8]:
if defined?(b) && b
  b
else
  b = calculate_initial_value
end


called: calculate_initial_value


false

In [9]:
a = 1
defined?(a)

"local-variable"

In [10]:
a = nil
defined?(a)

"local-variable"

In [11]:
p defined?(c)

nil


查看这个用`if`语句实现的空指针保护，会发现它实际上不能区分`false`和`nil`。在前一个例子中，`b`的值是`false`，因此空指针保护每次都对它进行初始化。

空指针保护的这个小问题一般不会被注意。但是有时它可能会导致不易追踪的`bug`。因此，不应该在变量值可能是`false`时对它进行空指针保护。

在Ruby中，经常可以看到长长的方法调用链，比如下面这样

In [12]:
['a', 'b', 'c'].push('d').shift.upcase.next

"B"

可以用`tap`方法在链的中间插入操作

In [14]:
['a', 'b', 'c'].push('d').shift.tap {|x| puts x}.upcase.next

a


"B"

In [15]:
class Object
  def tap
    yield self
    self
  end
end

:tap

In [16]:
['a', 'b', 'c'].push('d').shift.tap {|x| puts x}.upcase.next

a


"B"

In [17]:
require 'faraday'

conn = Faraday.new("https://twitter.com/search") do |faraday|
  faraday.response  :logger
  faraday.adapter   Faraday.default_adapter
  faraday.params["q"] = "ruby"
  faraday.params["src"] = "type"
end
  

#<Faraday::Connection:0x23d5ca0 @parallel_manager=nil, @headers={"User-Agent"=>"Faraday v0.11.0"}, @params={"q"=>"ruby", "src"=>"type"}, @options=#<Faraday::RequestOptions (empty)>, @ssl=#<Faraday::SSLOptions (empty)>, @default_parallel_manager=nil, @builder=#<Faraday::RackBuilder:0x23d5bf8 @handlers=[Faraday::Response::Logger, Faraday::Adapter::NetHttp]>, @url_prefix=#<URI::HTTPS https://twitter.com/search>, @proxy=nil>

In [18]:
response = conn.get
response.status

200

Rail4则更进一部；定义一个属性访问器时，会将它变成一个自由方法，并保存在方法缓存池中。如果第二个类具有同名属性，因而需要童谣的访问器时，Rails 4会从方法缓存池中取出前面定义的访问器，并将它绑定到第二个类上。通过这种方式，即使是无关的类，如果它们正好具有同名的属性，Rails也只会定义一组属性访问器，并对所有这些属性都使用这一组属性访问其。

尽量保持简单，只在必要时才增加复杂性。刚开始时，首先要保证代码正确，并且保持代码简单。这样遇到特例时才方便处理。

这条原则适用于绝大多数场合，尤其是在元编程里。

例如，像`private`和`protected`这样的访问修饰符都是拟态方法，像是`attr_reader`这样的类宏也是拟态方法

In [24]:
module ActiveRecord
  module AttributeMethods
    module Read
      ReaderMethodCache = Class.new(AttributeMethodCache) {
        private
        
        def method_body(method_name, const_name)
          <<-EOMETHOD
          def #{method_name}
            name = ::ActiveRecord::AttributeMethods::ATTR_#{const_name}
            read_attribute(name) {|n| missing_attribute(n, caller)}
            
          end
          EOMETHOD
        end
      }.new
    end
  end

SyntaxError: (pry):116: syntax error, unexpected end-of-input, expecting keyword_end

由于`define_method`在派发时速度较慢，我们想通过`module_eval`来创建方法。但如果`module_eval`创建了很多类似的方法，又会占用较多的内存，因为这些代码字符串会到处复制并缓存（在MRI中）。`define_method`方法尽管在派发时速度慢，但是，如果注意其生成的闭包，`define_method`占用的内存会比较小。

But sometimes the database might return columns with
characters that are not allowed in normal method names (like
my_column(omg)'. So to work around this we first define with
the __temp__ identifier, and then use alias method to rename
it to what we want.